Import Libraries

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
from keras.applications import mobilenet
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

Load Data

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
train_path = '/content/drive/MyDrive/data/train'
test_path = '/content/drive/MyDrive/data/test'
valid_path = '/content/drive/MyDrive/data/valid'

train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (224, 224), batch_size = 10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size = (224, 224), batch_size = 10)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size = (224, 224), batch_size = 10, shuffle = False)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


Load pre-trained VGG16's weights

In [22]:
base_model = vgg16.VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

Freeze the weights of the convolutional layers

In [23]:
for layer in base_model.layers:
  layer.trainable = False

Add classification layers / Define New Model

In [24]:
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output

x = Flatten()(last_output)

x = Dense(64, activation = 'relu', name = 'FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(2, activation = 'softmax', name = 'softmax')(x)

new_model = Model(inputs = base_model.input, outputs = x)
new_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC_2 (Dense)                    │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 16,320,770 (62.26 MB)

 Trainable params: 1,605,954 (6.13 MB)

 Non-trainable params: 14,714,816 (56.13 MB)

 Compile and Train the Model

In [25]:
new_model.compile(Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

new_model.fit(train_batches, steps_per_epoch = 4, validation_data = valid_batches, validation_steps = 2, epochs = 20, verbose = 2)

Epoch 1/20
4/4 - 46s - 11s/step - accuracy: 0.5000 - loss: 1.0942 - val_accuracy: 0.7500 - val_loss: 1.7543
Epoch 2/20
4/4 - 41s - 10s/step - accuracy: 0.6000 - loss: 1.0073 - val_accuracy: 0.6000 - val_loss: 1.3941
Epoch 3/20
4/4 - 42s - 11s/step - accuracy: 0.7188 - loss: 0.8340 - val_accuracy: 0.7500 - val_loss: 1.3927
Epoch 4/20
4/4 - 41s - 10s/step - accuracy: 0.9000 - loss: 0.3417 - val_accuracy: 0.8000 - val_loss: 0.8546
Epoch 5/20
4/4 - 81s - 20s/step - accuracy: 0.8250 - loss: 0.4724 - val_accuracy: 0.9000 - val_loss: 0.5811
Epoch 6/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


4/4 - 18s - 5s/step - accuracy: 0.9000 - loss: 0.2640 - val_accuracy: 0.6500 - val_loss: 0.6255
Epoch 7/20
4/4 - 143s - 36s/step - accuracy: 0.8125 - loss: 0.3565 - val_accuracy: 0.9000 - val_loss: 0.2771
Epoch 8/20
4/4 - 34s - 8s/step - accuracy: 0.7500 - loss: 0.5387 - val_accuracy: 0.7500 - val_loss: 0.6284
Epoch 9/20
4/4 - 33s - 8s/step - accuracy: 0.9750 - loss: 0.1102 - val_accuracy: 0.8000 - val_loss: 0.2241
Epoch 10/20
4/4 - 35s - 9s/step - accuracy: 1.0000 - loss: 0.0892 - val_accuracy: 0.6500 - val_loss: 0.6694
Epoch 11/20
4/4 - 36s - 9s/step - accuracy: 0.9000 - loss: 0.2182 - val_accuracy: 0.8000 - val_loss: 0.2143
Epoch 12/20
4/4 - 17s - 4s/step - accuracy: 1.0000 - loss: 0.1554 - val_accuracy: 0.8000 - val_loss: 0.4852
Epoch 13/20
4/4 - 82s - 20s/step - accuracy: 0.9500 - loss: 0.2263 - val_accuracy: 0.8500 - val_loss: 0.3712
Epoch 14/20
4/4 - 41s - 10s/step - accuracy: 0.8750 - loss: 0.6867 - val_accuracy: 0.9000 - val_loss: 0.2057
Epoch 15/20
4/4 - 40s - 10s/step - accu

Evaluate the model

In [25]:
from sklearn.datasets import load_files
from keras.utils import up_utils
import numpy as np

def load_dataset(path):
  data = load_files(path)
  files = np.array(data['filenames'])
  targets = up_utils.to_categorical(np.array(data['target']))
  return files, targets

test_files, test_targets = load_dataset(test_path)

Test data to tensor

In [32]:
import os

test_files = [os.path.join(test_path, fname) for fname in test_batches.filenames]

In [33]:
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tqdm import tqdm

def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    return x

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(p) for p in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))



  0%|          | 0/451 [00:00<?, ?it/s]

  0%|          | 1/451 [00:00<01:32,  4.88it/s]

  0%|          | 2/451 [00:00<01:23,  5.40it/s]

  1%|          | 3/451 [00:00<01:49,  4.09it/s]

  1%|          | 4/451 [00:00<01:35,  4.69it/s]

  1%|          | 5/451 [00:01<01:41,  4.41it/s]

  1%|▏         | 6/451 [00:01<01:37,  4.57it/s]

  2%|▏         | 7/451 [00:01<01:42,  4.35it/s]

  2%|▏         | 8/451 [00:01<01:40,  4.42it/s]

  2%|▏         | 9/451 [00:02<01:52,  3.92it/s]

  2%|▏         | 10/451 [00:02<01:48,  4.05it/s]

  2%|▏         | 11/451 [00:02<02:06,  3.47it/s]

  3%|▎         | 12/451 [00:03<02:32,  2.88it/s]

  3%|▎         | 13/451 [00:03<02:32,  2.87it/s]

  3%|▎         | 14/451 [00:03<02:12,  3.31it/s]

  3%|▎         | 15/451 [00:04<03:16,  2.22it/s]

  4%|▎         | 16/451 [00:04<03:09,  2.30it/s]

  4%|▍         | 17/451 [00:05<02:39,  2.72it/s]

  4%|▍         | 18/451 [00:05<02:15,  3.19it/s]

  4%|▍         | 19/451 [00:05<01:58,  3.66it/s]

  4%|▍         |

Check the Accuracy

In [35]:
from tensorflow.keras.utils import to_categorical

test_targets = to_categorical(test_batches.classes)

In [36]:
print('\nloss function: {:.4f}\naccuracy:{:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

  0%|          | 0/451 [03:25<?, ?it/s]


15/15 ━━━━━━━━━━━━━━━━━━━━ 242s 16s/step - accuracy: 0.9473 - loss: 0.1174

loss function: 0.1387
accuracy:0.9401
